<a href="https://colab.research.google.com/github/jinhyung426/tensorflow/blob/main/tf_chap1_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow

## Part 1. Introduction to TensorFlow for Artificial Intelligence, Machine Learning, and Deep Learning


In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Callbacks

1. Inline으로 해도 가능
2. on_epoch_end function의 경우 epoch가 끝나고 나서 실행
3. logs.get('loss'), logs.get('mae') 등 loss 및 metrics로 설정한 것들 확인할 수 있음
4. For lower versions of tensorflow, use 'acc' instead of 'accuracy'

In [8]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if epoch.get('accuracy') > 0.82:
      print("Reahced 82% accuracy so stop training!\n")
      self.model.stop_training = True

In [9]:
callback = myCallback()

In [11]:
mnist = tf.keras.datasets.fashion_mnist  # fashion_mnist는 옷, mnist는 손글씨 숫자 data

(training_images, training_labels), (testing_images, testing_labels) = mnist.load_data()
training_images = training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0
testing_images = testing_images.reshape(10000, 28, 28, 1)
testing_images = testing_images / 255.0

4423680/4422102 [==============================] - 0s 0us/step
